### Imports

In [20]:
import pandas as pd
from pycaret.regression import *
from sklearn.model_selection import train_test_split
import h2o
from h2o.automl import H2OAutoML
import pickle
from datetime import timedelta
import talib
import numpy as np

In [21]:
# Evito que ciertas columnas se transformen a notacion cientifica en las predicciones
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)

### Dataset

In [22]:
columns = [
    'Open_time',
    'Open',
    'High',
    'Low',
    'Close',
    'Number of trades',
    'Close_BTCUSDT',
    'Volume_BTCUSDT',
    'Number_of_trades_BTCUSDT',
    'Close_ETHUSDT',
    'Volume_ETHUSDT',
    'Number_of_trades_ETHUSDT',
    'Close_BNBUSDT',
    'Volume_BNBUSDT',
    'Number_of_trades_BNBUSDT',
    'SMA_20',
    'EMA_20',
    'Upper_Band',
    'Middle_Band',
    'Lower_Band',
    'RSI',
    'buy_1000x_high_coinbase',
    'sell_1000x_high_coinbase',
    'total_trades_coinbase',	
    'Tweets_Utilizados',
    'Tweets_Utilizados_coin',
    'Tweets_Utilizados_referentes',
    'Tweets_Utilizados_whale_alert',
    'Buy_1000x_high',
    'sell_1000x_high',
    'total_trades_binance'
]

columns_to_model = [
    'Open',
    'High',
    'Low',
    'Close',
    'Number of trades',
    'Close_BTCUSDT',
    'Volume_BTCUSDT',
    'Number_of_trades_BTCUSDT',
    'Close_ETHUSDT',
    'Volume_ETHUSDT',
    'Number_of_trades_ETHUSDT',
    'Close_BNBUSDT',
    'Volume_BNBUSDT',
    'Number_of_trades_BNBUSDT',
    # 'SMA_20',
    # 'EMA_20',
    # 'Upper_Band',
    # 'Middle_Band',
    # 'Lower_Band',
    # 'RSI',
    'buy_1000x_high_coinbase',
    'sell_1000x_high_coinbase',
    'total_trades_coinbase',	
    'Tweets_Utilizados',
    'Tweets_Utilizados_coin',
    'Tweets_Utilizados_referentes',
    'Tweets_Utilizados_whale_alert',
    'Buy_1000x_high',
    'sell_1000x_high',
    'total_trades_binance'
]

In [23]:
dataset = pd.read_csv('/Users/mmarchetta/Desktop/Tesis-2024/data-visualization/final_dataset.csv') 
dataset['Open_time'] = pd.to_datetime(dataset['Open_time'])

display(dataset.head())

,Open_time,Open,High,Low,Close,Volume,Quote asset volume,Number of trades,Open_BTCUSDT,High_BTCUSDT,Low_BTCUSDT,Close_BTCUSDT,Volume_BTCUSDT,Quote_asset_volume_BTCUSDT,Number_of_trades_BTCUSDT,Open_ETHUSDT,High_ETHUSDT,Low_ETHUSDT,Close_ETHUSDT,Volume_ETHUSDT,Quote_asset_volume_ETHUSDT,Number_of_trades_ETHUSDT,Open_BNBUSDT,High_BNBUSDT,Low_BNBUSDT,Close_BNBUSDT,Volume_BNBUSDT,Quote_asset_volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,MACD,Signal,ADX,SlowK,SlowD,CCI,ATR,MFI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Sentimiento,Tweets_Utilizados,Cantidad_pos,Cantidad_neg,Cantidad_neu,Compound total,Sentimiento_coin,Tweets_Utilizados_coin,Cantidad_pos_coin,Cantidad_neg_coin,Cantidad_neu_coin,Compound total coin,Sentimiento_referentes,Tweets_Utilizados_referentes,Cantidad_pos_referentes,Cantidad_neg_referentes,Cantidad_neu_referentes,Compound total referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance,Tendencia
0,2021-09-27,28.84,30.26,27.50,27.71,11094418.10,320340017.82,449178.00,43160.90,44350.00,42098.00,42147.35,39776.84,1728912299.80,1001487.00,3059.36,3164.94,2922.71,2925.59,510130.73,1564096695.37,1043885.00,344.00,357.60,335.00,335.50,956544.07,332372476.51,457187.00,31.85,30.83,38.30,31.85,25.39,44.11,0.02,0.96,21.20,24.79,33.78,-86.30,4.00,29.23,0.00,0.00,0.00,pos,139,61.00,28.00,50.00,17.83,pos,135,71.00,26.00,38.00,30.62,neu,1.00,0.00,0.00,1.00,0.00,81.00,525.00,364.00,270000.00,Lateral
1,2021-09-28,27.72,28.38,26.14,26.31,9784154.53,267560973.69,362304.00,42147.35,42787.38,40888.00,41026.54,43372.26,1816305387.84,1045389.00,2924.76,2970.82,2789.00,2804.91,511325.46,1477983245.67,928494.00,335.50,344.60,330.00,333.00,922077.23,310828343.70,417006.00,31.77,30.40,38.44,31.77,25.11,41.83,-0.33,0.70,21.98,16.10,24.47,-109.46,3.88,19.19,4.00,2.00,5273.00,pos,93,40.00,21.00,32.00,10.18,pos,122,57.00,23.00,42.00,19.89,neu,2.00,1.00,1.00,0.00,0.02,87.00,472.00,331.00,204000.00,Bajista
2,2021-09-29,26.31,28.59,26.11,27.28,10207287.33,280535376.38,376232.00,41025.01,42590.00,40753.88,41524.28,33511.53,1399625476.06,884909.00,2805.15,2950.00,2782.00,2850.45,411305.09,1180681056.16,748804.00,332.90,375.40,331.20,367.70,1696420.04,612547661.88,653011.00,31.65,30.10,38.55,31.65,24.74,43.99,-0.52,0.46,22.61,12.88,17.92,-92.93,3.78,23.82,22.00,40.00,54144.00,pos,112,61.00,19.00,32.00,23.13,pos,145,71.00,20.00,54.00,30.30,neu,0.00,0.00,0.00,0.00,0.00,64.00,594.00,495.00,216000.00,Alcista
3,2021-09-30,27.28,28.99,27.13,28.62,8201551.44,230622998.43,339737.00,41524.29,44141.37,41410.17,43824.10,46381.23,2005486003.56,1197815.00,2850.45,3048.42,2837.19,3000.61,506896.76,1514341937.31,992243.00,367.70,388.80,366.40,387.50,1163674.21,439628574.42,551245.00,31.62,29.96,38.56,31.62,24.67,46.92,-0.56,0.26,22.99,16.42,15.13,-60.03,3.64,28.34,15.00,23.00,39220.00,pos,116,46.00,28.00,42.00,7.68,pos,147,73.00,26.00,48.00,30.90,pos,2.00,1.00,0.00,1.00,0.75,77.00,419.00,464.00,202000.00,Alcista
4,2021-10-01,28.61,32.33,28.50,31.94,13176348.21,402700970.65,735059.00,43820.01,48495.00,43283.03,48141.61,66244.87,3076094886.59,1771237.00,3000.62,3330.00,2969.07,3309.91,648714.62,2063112280.67,1446386.00,387.50,423.40,381.50,421.50,1440336.04,584311192.72,727854.00,31.64,30.15,38.58,31.64,24.69,53.42,-0.31,0.14,21.86,34.90,21.40,34.17,3.65,35.02,24.00,35.00,63183.00,pos,171,72.00,21.00,78.00,30.72,pos,141,70.00,18.00,53.00,30.43,pos,1.00,1.00,0.00,0.00,0.82,71.00,477.00,664.00,492000.00,Alcista


In [24]:
n_days_to_predict = 10

dates = dataset['Open_time'][:-n_days_to_predict]

dataset = dataset.round(2)

feature_dataset = dataset[columns]
# feature_dataset['Open_time'] = dataset['Open_time']
# feature_dataset.set_index('Open_time', inplace=True)

validation = feature_dataset[-n_days_to_predict:]
feature_dataset = feature_dataset[:-n_days_to_predict]

In [25]:
display(feature_dataset.head())
print(feature_dataset.shape)

display(validation.head())
display(validation.shape)

,Open_time,Open,High,Low,Close,Number of trades,Close_BTCUSDT,Volume_BTCUSDT,Number_of_trades_BTCUSDT,Close_ETHUSDT,Volume_ETHUSDT,Number_of_trades_ETHUSDT,Close_BNBUSDT,Volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Tweets_Utilizados_coin,Tweets_Utilizados_referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance
0,2021-09-27,28.84,30.26,27.50,27.71,449178.00,42147.35,39776.84,1001487.00,2925.59,510130.73,1043885.00,335.50,956544.07,457187.00,31.85,30.83,38.30,31.85,25.39,44.11,0.00,0.00,0.00,139,135,1.00,81.00,525.00,364.00,270000.00
1,2021-09-28,27.72,28.38,26.14,26.31,362304.00,41026.54,43372.26,1045389.00,2804.91,511325.46,928494.00,333.00,922077.23,417006.00,31.77,30.40,38.44,31.77,25.11,41.83,4.00,2.00,5273.00,93,122,2.00,87.00,472.00,331.00,204000.00
2,2021-09-29,26.31,28.59,26.11,27.28,376232.00,41524.28,33511.53,884909.00,2850.45,411305.09,748804.00,367.70,1696420.04,653011.00,31.65,30.10,38.55,31.65,24.74,43.99,22.00,40.00,54144.00,112,145,0.00,64.00,594.00,495.00,216000.00
3,2021-09-30,27.28,28.99,27.13,28.62,339737.00,43824.10,46381.23,1197815.00,3000.61,506896.76,992243.00,387.50,1163674.21,551245.00,31.62,29.96,38.56,31.62,24.67,46.92,15.00,23.00,39220.00,116,147,2.00,77.00,419.00,464.00,202000.00
4,2021-10-01,28.61,32.33,28.50,31.94,735059.00,48141.61,66244.87,1771237.00,3309.91,648714.62,1446386.00,421.50,1440336.04,727854.00,31.64,30.15,38.58,31.64,24.69,53.42,24.00,35.00,63183.00,171,141,1.00,71.00,477.00,664.00,492000.00


(946, 31)


,Open_time,Open,High,Low,Close,Number of trades,Close_BTCUSDT,Volume_BTCUSDT,Number_of_trades_BTCUSDT,Close_ETHUSDT,Volume_ETHUSDT,Number_of_trades_ETHUSDT,Close_BNBUSDT,Volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Tweets_Utilizados_coin,Tweets_Utilizados_referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance
946,2024-04-30,6.59,6.67,6.04,6.42,115512.00,60672.00,54947.66,1985671.00,3014.05,561717.49,1292873.00,578.40,766513.45,486465.00,6.93,7.13,7.80,6.93,6.05,34.18,51.00,55.00,48709.00,142,187,1.00,23.00,379.00,377.00,70000.00
947,2024-05-01,6.42,6.94,6.13,6.90,175570.00,58364.97,81166.47,2401089.00,2972.46,624963.78,1365039.00,561.80,669027.32,427425.00,6.85,7.11,7.41,6.85,6.29,43.30,42.00,50.00,83718.00,130,177,0.00,36.00,327.00,340.00,107000.00
948,2024-05-02,6.90,7.41,6.69,7.27,109002.00,59060.61,47583.82,1572898.00,2986.19,365939.72,880167.00,560.50,359794.32,250921.00,6.85,7.12,7.42,6.85,6.29,49.27,87.00,57.00,61208.00,461,374,1.00,25.00,270.00,282.00,71000.00
949,2024-05-03,7.27,7.39,7.00,7.24,109634.00,62882.01,43628.40,1558661.00,3102.61,355825.84,859542.00,587.00,342906.43,257575.00,6.90,7.14,7.44,6.90,6.35,48.86,52.00,40.00,46255.00,573,474,1.00,22.00,386.00,635.00,69000.00
950,2024-05-04,7.24,7.28,7.10,7.12,71120.00,63892.04,24368.69,1113509.00,3117.23,196263.95,575026.00,585.70,197129.25,210303.00,6.91,7.13,7.46,6.91,6.36,46.98,68.00,50.00,34251.00,407,472,0.00,14.00,203.00,232.00,49000.00


(10, 31)

### Model

#### Generacion de los modelos

In [26]:
# train_size = 0.8
# train_data, test_data = train_test_split(feature_dataset, train_size=train_size, shuffle=False)

In [27]:
h2o.init()

selected_models = {}

for column in columns_to_model:
    
    # A cada registro, le asigno como target el valor de Column del dia siguiente,
    # de esta forma, los N modelos son entrenados para predecir el valor de los N
    # atributos en un dia siguiente.
    training_data = feature_dataset
    training_data['Next_Day_Target'] = feature_dataset[column].shift(-1)
    training_data = training_data[:-1] #Tengo que sacar el ultimo registro porque no tendra valor del target
    ##
    h2o_df = h2o.H2OFrame(training_data.drop(columns=["Open_time"]))

    aml = H2OAutoML(seed=1234) #TODO: SACAR EL max_models
    aml.train(y='Next_Day_Target', training_frame=h2o_df)

    leaderboard = aml.leaderboard
    print(leaderboard)

    top_models = leaderboard.head(3)["model_id"].as_data_frame()["model_id"].tolist()

    best_model_id = top_models[0]
    best_model = h2o.get_model(best_model_id)
    
    selected_models[column] = best_model

Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,3 hours 39 mins
H2O_cluster_timezone:,America/Argentina/Buenos_Aires
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,1 month and 27 days
H2O_cluster_name:,H2O_from_python_mmarchetta_yyk9jw
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,6.876 Gb
H2O_cluster_total_cores:,16
H2O_cluster_allowed_cores:,16
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
model_id                                                       rmse         mse        mae       rmsle    mean_residual_deviance
StackedEnsemble_BestOfFamily_4_AutoML_25_20240511_134718  0.0892722  0.00796953  0.0656247  0.00853769                0.00796953
DeepLearning_grid_1_AutoML_25_20240511_134718_model_4     0.116454   0.0135616   0.0870703  0.011697                  0.0135616
StackedEnsemble_AllModels_2_AutoML_25_20240511_134718     0.155455   0.0241663   0.0915821  0.00913311                0.0241663
StackedEnsemble_AllModels_1_AutoML_25_20240511_134718     0.156245   0.0244126   0.0908327  0.00916294                0.0244126
GBM_grid_1_AutoML_25_20240511_134718_model_473            0.156669   0.0245453   0.0517629  0.00567586                0.0245453
StackedEnsemble_BestOfFamily_2_AutoML_25

In [28]:
predictions_dict = {}

for column, model in selected_models.items():
    predictions = model.predict(h2o.H2OFrame(validation.drop(columns=["Open_time"])))
    predictions_dict[column] = predictions

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |█████████████████████████████████

In [29]:
for column, predictions in predictions_dict.items():
    print(f"Predicciones para los próximos días de {column}:")
    print(predictions)

Predicciones para los próximos días de Open:
  predict
  6.51632
  7.01291
  7.35297
  7.28706
  7.17292
  7.34477
  7.16688
  7.05523
  7.06082
  7.11525
[10 rows x 1 column]

Predicciones para los próximos días de High:
  predict
  6.514
  6.98693
  7.25697
  7.50944
  7.16125
  7.1489
  7.00151
  6.9728
  6.92004
  7.07034
[10 rows x 1 column]

Predicciones para los próximos días de Low:
  predict
  6.16707
  6.37333
  6.80107
  7.07104
  6.91066
  6.96066
  6.88863
  6.79779
  6.7659
  6.78159
[10 rows x 1 column]

Predicciones para los próximos días de Close:
  predict
  6.52459
  6.86868
  7.09519
  7.29609
  7.2142
  7.13806
  7.14502
  7.18787
  6.96238
  6.90535
[10 rows x 1 column]

Predicciones para los próximos días de Number of trades:
  predict
 109136
 154824
 129294
 143412
  90359.3
  73823.2
  97791.3
  78793.2
  90359.3
  76724.7
[10 rows x 1 column]

Predicciones para los próximos días de Close_BTCUSDT:
  predict
  62001.3
  59819
  59478
  62495.8
  63688.4
  63412

#### Guardo el mejor modelo para cada columna

In [30]:
for column, model in selected_models.items():
    file_name = f"modelo_{column}.h2o"
    h2o.save_model(model, 'h2o_models', filename=file_name)

#### Vuelvo a armar el mejor modelo para cada columna

In [31]:
# h2o.init()

In [32]:
loaded_models = {}
for column in columns_to_model:
    model_path = f"modelo_{column}.h2o"
    loaded_models[column] = h2o.load_model(f"h2o_models/{model_path}")

#### Prediccion 1 dia en el futuro

In [33]:
def hacer_predicciones_para_dia_siguiente(trained_models, last_day_data):
    """
    Predice los valores para el día siguiente utilizando los modelos entrenados y el último registro de datos.

    Args:
    - trained_models: Un diccionario donde las claves son las características y los valores son los modelos entrenados.
    - last_day_data: Pandas Series que representa el último registro de datos con todas las características excepto la variable objetivo.

    Returns:
    - Pandas DataFrame con un solo registro que contiene las predicciones para el día siguiente.
    """
    # Crear un diccionario para almacenar las predicciones
    predictions = {}

    # Hacer predicciones para cada característica utilizando los modelos entrenados
    for feature, model in trained_models.items():
        # Preparar los datos para la predicción (convertir a DataFrame de H2O)
        h2o_last_day_data = h2o.H2OFrame(last_day_data)
        
        # Realizar la predicción para la característica actual
        prediction = model.predict(h2o_last_day_data).as_data_frame().iloc[0, 0]
        
        # Almacenar la predicción en el diccionario
        predictions[feature] = prediction

    # Convertir el diccionario de predicciones en un DataFrame de pandas
    predictions_df = pd.DataFrame(predictions, index=[0])

    return predictions_df

In [34]:
predictions_next_day = hacer_predicciones_para_dia_siguiente(loaded_models, feature_dataset.drop(columns=["Open_time", "Next_Day_Target"])[-1:])

# Muestra el DataFrame con las predicciones para el día siguiente
display(predictions_next_day)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |█████████████████████████████████

,Open,High,Low,Close,Number of trades,Close_BTCUSDT,Volume_BTCUSDT,Number_of_trades_BTCUSDT,Close_ETHUSDT,Volume_ETHUSDT,Number_of_trades_ETHUSDT,Close_BNBUSDT,Volume_BNBUSDT,Number_of_trades_BNBUSDT,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Tweets_Utilizados_coin,Tweets_Utilizados_referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance
0,6.65,6.63,6.29,6.52,90308.97,63385.62,22791.41,1387535.49,3202.72,366273.31,926573.93,585.94,602299.57,306200.58,43.60,41.23,37739.58,128.52,121.28,0.22,24.06,243.07,249.28,40712.76


#### Prediccion de los N dias en el futuro

In [35]:
def calcular_indicadores_tecnicos(historico_precio):
    close_prices = np.array(historico_precio['Close'], dtype=np.double)
    # high_prices = np.array(historico_precio['High'], dtype=np.double)
    # low_prices = np.array(historico_precio['Low'], dtype=np.double)

    # Media Móvil Simple (SMA)
    sma_20 = talib.SMA(close_prices, timeperiod=20)

    # Media Móvil Exponencial (EMA)
    ema_20 = talib.EMA(close_prices, timeperiod=20)

    # Bandas de Bollinger
    upper_band, middle_band, lower_band = talib.BBANDS(close_prices, timeperiod=20, nbdevup=2, nbdevdn=2)

    # Índice de Fuerza Relativa (RSI)
    rsi = talib.RSI(close_prices, timeperiod=14)

    # Crear un DataFrame para almacenar los indicadores técnicos
    indicadores = pd.DataFrame({
        'Open_time': historico_precio['Open_time'],
        'SMA_20': sma_20,
        'EMA_20': ema_20,
        'Upper_Band': upper_band,
        'Middle_Band': middle_band,
        'Lower_Band': lower_band,
        'RSI': rsi
    }, index=historico_precio.index)

    return indicadores

In [36]:
for i in range(n_days_to_predict):
    predictions_next_day = hacer_predicciones_para_dia_siguiente(loaded_models, feature_dataset.drop(columns=["Open_time", "Next_Day_Target"])[-1:])
    
    ultima_fecha = feature_dataset["Open_time"].iloc[-1]
    nueva_fecha = ultima_fecha + timedelta(days=1)
    
    nuevo_registro = pd.DataFrame({"Open_time": [nueva_fecha]})

    for column, prediction in predictions_next_day.items():
        nuevo_registro[column] = prediction
        
    feature_dataset = pd.concat([feature_dataset, nuevo_registro], ignore_index=True)

    nuevos_indicadores = calcular_indicadores_tecnicos(feature_dataset)
    
    for column in nuevos_indicadores.columns:
        feature_dataset.loc[feature_dataset.index[-1], column] = nuevos_indicadores.loc[nuevos_indicadores.index[-1], column]
    
    # display(nuevos_indicadores)mm
    

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%
Parse progress: |█████████████████████████████████

In [37]:
display(feature_dataset.tail(n_days_to_predict + 5))

,Open_time,Open,High,Low,Close,Number of trades,Close_BTCUSDT,Volume_BTCUSDT,Number_of_trades_BTCUSDT,Close_ETHUSDT,Volume_ETHUSDT,Number_of_trades_ETHUSDT,Close_BNBUSDT,Volume_BNBUSDT,Number_of_trades_BNBUSDT,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,buy_1000x_high_coinbase,sell_1000x_high_coinbase,total_trades_coinbase,Tweets_Utilizados,Tweets_Utilizados_coin,Tweets_Utilizados_referentes,Tweets_Utilizados_whale_alert,Buy_1000x_high,sell_1000x_high,total_trades_binance,Next_Day_Target
941,2024-04-25,6.93,7.00,6.70,6.86,71088.00,64498.34,31341.46,1375324.00,3155.80,352288.55,861077.00,613.20,453745.52,353114.00,7.43,7.45,9.08,7.43,5.77,38.83,21.00,26.00,33468.00,151.00,114.00,0.00,22.00,242.00,219.00,48000.00,42000.00
942,2024-04-26,6.86,6.95,6.71,6.76,67383.00,63770.01,27085.19,1025561.00,3131.30,252522.66,628635.00,598.00,302119.88,269508.00,7.34,7.38,8.94,7.34,5.74,37.81,29.00,24.00,26619.00,117.00,106.00,0.00,14.00,292.00,324.00,42000.00,41000.00
943,2024-04-27,6.76,6.87,6.51,6.81,64779.00,63461.98,20933.06,912422.00,3255.56,323811.19,734026.00,596.20,268783.91,233820.00,7.24,7.33,8.73,7.24,5.76,38.57,17.00,17.00,25565.00,101.00,138.00,0.00,7.00,248.00,179.00,41000.00,26000.00
944,2024-04-28,6.81,6.95,6.69,6.73,43208.00,63118.62,16949.20,790652.00,3263.45,304766.01,753239.00,600.20,258059.43,206703.00,7.13,7.27,8.38,7.13,5.88,37.66,16.00,20.00,20954.00,82.00,106.00,0.00,13.00,173.00,165.00,26000.00,41000.00
945,2024-04-29,6.73,6.83,6.47,6.59,63006.00,63866.00,28150.23,1152296.00,3216.73,421831.29,943719.00,592.80,330474.00,271926.00,7.03,7.20,8.08,7.03,5.97,36.02,69.00,37.00,33959.00,115.00,125.00,0.00,24.00,260.00,188.00,41000.00,NaN
946,2024-04-30,6.65,6.63,6.29,6.52,90308.97,63385.62,22791.41,1387535.49,3202.72,366273.31,926573.93,585.94,602299.57,306200.58,6.93,7.14,7.79,6.93,6.07,35.28,43.60,41.23,37739.58,128.52,121.28,0.22,24.06,243.07,249.28,40712.76,NaN
947,2024-05-01,6.59,6.63,6.32,6.61,100641.59,63417.44,26115.83,1506021.14,3187.77,394015.30,946105.28,581.62,559689.72,319150.46,6.84,7.09,7.40,6.84,6.28,37.20,40.87,38.73,39121.73,132.23,125.69,0.27,27.80,295.15,247.35,42742.03,NaN
948,2024-05-02,6.68,6.64,6.34,6.62,100641.59,63255.23,26287.19,1448864.11,3183.99,365609.52,987325.95,580.98,546964.29,337461.16,6.81,7.04,7.34,6.81,6.28,37.41,41.80,38.59,38531.05,122.41,128.86,0.21,28.04,274.50,260.73,45336.30,NaN
949,2024-05-03,6.69,6.69,6.35,6.65,100641.59,63240.42,26079.18,1483258.15,3175.34,358810.91,1036213.97,579.59,557270.78,326291.10,6.82,7.01,7.33,6.82,6.32,38.36,41.70,39.18,39377.53,122.13,132.62,0.20,25.97,274.72,252.38,33310.89,NaN
950,2024-05-04,6.73,6.73,6.41,6.70,102581.49,63318.04,27314.36,1527027.84,3172.58,378339.91,1063270.13,579.43,565714.92,309541.86,6.82,6.98,7.32,6.82,6.31,39.40,41.63,40.26,41159.15,127.59,124.38,0.21,26.49,284.79,257.74,48445.24,NaN


In [38]:
ultimos_registros = feature_dataset.tail(n_days_to_predict)
ultimos_registros.to_csv('h2o_models/predicciones.csv', index=False) 

Si predice maso menos bien para estas variables hacer lo siguiente:
- Evaluar los resultados por columna (definir alguna forma de hacer esto)

- Armar el clasificador (LSTM o CONV), entrenandolo sobre los datos del dataset original ( sin los ultimos 5 registros)
- Probar como predice el target (Tendencia) para los ultimos 5 registros con los datos generados por los modelos